In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


Based on https://arxiv.org/abs/1409.0473

In [2]:
using Flux: flip, crossentropy, reset!, throttle

include("0-data.jl")

Nin = length(alphabet)
Nh = 30 # size of hidden layer

30

A recurrent model which takes a token and returns a context-dependent
annotation.

In [3]:
forward  = LSTM(Nin, Nh÷2)
backward = LSTM(Nin, Nh÷2)
encode(tokens) = vcat.(forward.(tokens), flip(backward, tokens))

alignnet = Dense(2Nh, 1)
align(s, t) = alignnet(vcat(t, s .* trues(1, size(t, 2))))

align (generic function with 1 method)

A recurrent model which takes a sequence of annotations, attends, and returns
a predicted output token.

In [4]:
recur   = LSTM(Nh+length(phones), Nh)
toalpha = Dense(Nh, length(phones))

function asoftmax(xs)
  xs = [exp.(x) for x in xs]
  s = sum(xs)
  return [x ./ s for x in xs]
end

function decode1(tokens, phone)
  weights = asoftmax([align(recur.state[2], t) for t in tokens])
  context = sum(map((a, b) -> a .* b, weights, tokens))
  y = recur(vcat(float(phone), context))
  return softmax(toalpha(y))
end

decode(tokens, phones) = [decode1(tokens, phone) for phone in phones]

decode (generic function with 1 method)

The full model

In [5]:
state = (forward, backward, alignnet, recur, toalpha)

function model(x, y)
  ŷ = decode(encode(x), y)
  reset!(state)
  return ŷ
end

loss(x, yo, y) = sum(crossentropy.(model(x, yo), y))

evalcb = () -> @show loss(data[500]...)
opt = ADAM(params(state))

Flux.train!(loss, data, opt, cb = throttle(evalcb, 10))

loss(data[500]...) = 31.130797001386394 (tracked)
loss(data[500]...) = 19.542230529488833 (tracked)
loss(data[500]...) = 17.789988431246474 (tracked)
loss(data[500]...) = 16.552711488292868 (tracked)
loss(data[500]...) = 15.749679743532923 (tracked)
loss(data[500]...) = 15.051589664736781 (tracked)
loss(data[500]...) = 14.296520490145966 (tracked)
loss(data[500]...) = 14.5884759363154 (tracked)
loss(data[500]...) = 14.076095235362706 (tracked)
loss(data[500]...) = 13.570025356069744 (tracked)
loss(data[500]...) = 12.99752185470236 (tracked)
loss(data[500]...) = 12.641269158867225 (tracked)
loss(data[500]...) = 13.571412206376001 (tracked)
loss(data[500]...) = 13.627850846252963 (tracked)
loss(data[500]...) = 12.991679974109383 (tracked)
loss(data[500]...) = 12.70220467164414 (tracked)
loss(data[500]...) = 12.990818785153746 (tracked)
loss(data[500]...) = 15.265261599610595 (tracked)
loss(data[500]...) = 13.866875815496153 (tracked)
loss(data[500]...) = 14.363912225025175 (tracked)
loss

Prediction

In [6]:
using StatsBase: wsample

function predict(s)
  ts = encode(tokenise(s, alphabet))
  ps = Any[:start]
  for i = 1:50
    dist = decode1(ts, onehot(ps[end], phones))
    next = wsample(phones, vec(Tracker.data(dist)))
    next == :end && break
    push!(ps, next)
  end
  return ps[2:end]
end

predict("PHYLOGENY")

12-element Array{Any,1}:
 :TH 
 :ER1
 :L  
 :AE2
 :Z  
 :AH0
 :N  
 :OW1
 :D  
 :IH0
 :N  
 :G  